<a href="https://colab.research.google.com/github/tzf101/BDA-Bangla-Text-Data-Augmentation/blob/main/utils_notebook/mlm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Libraries



### Libraries

In [ ]:
from google.colab import drive
drive.mount("/content/MyDrive", force_remount=True)

Mounted at /content/MyDrive


In [ ]:
from google.colab import drive

In [ ]:
!pip install transformers

In [ ]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-999spavb
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-999spavb
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=a8eefde770a121dd8372992cf7f64f1c1fdd800443de1fa3b2aa6b79aa09282d
  Stored in directory: /tmp/pip-ephem-wheel-cache-v4jm5gzj/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186460 sha256=7327bea01ae27b5ac941c57643d657977360d91

In [ ]:
!pip install sentencepiece
!pip install rouge
!pip install sacrebleu
!pip install -U sentence-transformers
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1fd68985bdd86392837e63848106eb717992fc7c64856e62453b2f175afc3880
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.1 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from normalizer import normalize
import sentencepiece as spm
import pandas as pd

import sacrebleu
from rouge import Rouge
from sacrebleu import corpus_bleu
from bert_score import score
from sentence_transformers import SentenceTransformer, util
from nltk.translate.bleu_score import sentence_bleu

### Loading Models

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/banglabert_generator")
model = AutoModelForMaskedLM.from_pretrained("csebuetnlp/banglabert_generator")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/623 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/528k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/140M [00:00<?, ?B/s]

In [ ]:
sbert_model = SentenceTransformer('l3cube-pune/bengali-sentence-bert-nli')

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/6.94k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/519 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

### Score calculation

In [ ]:
def calculate_sbert_score(original, augmented):
    emb1 = sbert_model.encode(original)
    emb2 = sbert_model.encode(augmented)
    cosine_scores = util.pytorch_cos_sim(emb1, emb2)
    sbert_score = cosine_scores.item()
    return sbert_score

In [ ]:
def calculate_scores(original, augmented):
    # BLEU Score
    reference = original
    candidate = augmented
    # bleu_score = sentence_bleu(reference, candidate)
    bleu_score = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(augmented, original)]
    # BERTScore
    P, R, F1 = score([augmented], [original], lang="bn", rescale_with_baseline=True)

    # SBERT Score with Cosine Similarity
    emb1 = sbert_model.encode(original)
    emb2 = sbert_model.encode(augmented)
    cosine_scores = util.pytorch_cos_sim(emb1, emb2)
    sbert_score = cosine_scores.item()

    return bleu_score, F1.item(), sbert_score

### Masking and Unmasking functions

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
import random

class BengaliMaskedLanguageModel:
    def __init__(self, model, tokenizer):
        self.tokenizer = tokenizer
        self.model = model

    def tokenize(self, text: str) -> torch.Tensor:
        return self.tokenizer.encode(text, return_tensors='pt')

    def predict_tokens(self, input_ids: torch.Tensor) -> torch.Tensor:
        with torch.no_grad():
            outputs = self.model(input_ids)
            return outputs[0]

    def find_masked_indices(self, input_ids: torch.Tensor) -> torch.Tensor:
        return (input_ids == self.tokenizer.mask_token_id).nonzero(as_tuple=True)[1]

    def replace_masked_tokens(self, input_ids: torch.Tensor, predictions: torch.Tensor, masked_indices: torch.Tensor) -> torch.Tensor:
        for idx in masked_indices:
            predicted_token_id = predictions[0, idx].topk(1).indices.item()
            input_ids[0, idx] = predicted_token_id
        return input_ids

    def unmask_text(self, masked_text: str) -> str:
        input_ids = self.tokenize(masked_text)
        predictions = self.predict_tokens(input_ids)
        masked_indices = self.find_masked_indices(input_ids)
        unmasked_input_ids = self.replace_masked_tokens(input_ids, predictions, masked_indices)
        return self.tokenizer.decode(unmasked_input_ids[0], skip_special_tokens=True)

    def random_masking(self, text: str, num_mask: int) -> str:
        token_ids = self.tokenizer.encode(text, return_tensors='pt')[0]
        special_token_ids = {self.tokenizer.cls_token_id, self.tokenizer.sep_token_id, self.tokenizer.mask_token_id}

        # Avoid masking special tokens
        indices_to_mask = random.sample([i for i in range(1, len(token_ids) - 1) if token_ids[i] not in special_token_ids], min(num_mask, len(token_ids) - 2))

        masked_token_ids = [self.tokenizer.mask_token_id if i in indices_to_mask else token_id for i, token_id in enumerate(token_ids)]
        return self.tokenizer.decode(masked_token_ids)

    def mask_and_unmask(self, text: str, num_mask: int = 3) -> str:
        masked_text = self.random_masking(text, num_mask)
        unmasked_text = self.unmask_text(masked_text)

        return unmasked_text




### Testing

In [ ]:
# Usage
mlm = BengaliMaskedLanguageModel(model, tokenizer)

text = "দুনিয়াতে সব চেয়ে কঠিন কাজ হচ্ছে নিজেকে সংশোধন করা আর সব চেয়ে সহজ কাজ হচ্ছে অন্যের সমালোচনা করা"
text = "এগিয়ে যাও আমরা আছি তোমাদের সাথে"

num_mask = 2  # Number of words to mask

masked_text = mlm.random_masking(text, num_mask)
print(masked_text)

unmasked_text = mlm.unmask_text(masked_text)
print(unmasked_text)

[CLS] [UNK] যাও আমরা [MASK] তোমাদের [MASK] [SEP]
যাও আমরা সবাই তোমাদের কাছে


In [ ]:
output = mlm.mask_and_unmask(text)
print(output)

একজন ভাই একটু ত্যানাপ্যাচাল করছে, কোন প্রকার ad comment করবেন না । বাপের লাখ টাকা অার স্যারদের সাজেসন্সে না । নিজে কষ্ট করে করে শেষ করছি । হ্যা অামি মানি যারা পাবলিকে চান্স বা অপেক্ষা থেকে বাদ, তারাই তে, যাদের মেধা অাছে, অার্থিক অবস্থা ভালো না, বা কেউ যেনো কোনদিন বলতে না পারে বাপের টাকা । প্রতিযোগিতা করতে হবে । রিংকু সাহা ঢাকা কলেজ ২০১০ - ১১ হিঃবি


### Apply MLM on dataset

In [ ]:
# Define a function to apply paraphrasing
def apply_mlm(row):
    return mlm.mask_and_unmask(row['original_sentence'])

In [ ]:
def mlm_and_evaluate_dataset(file_path, original_col_name, new_col_name):
    # Load the dataset
    df = pd.read_csv(file_path)
    df = df.rename(columns={original_col_name: 'original_sentence'})

    # Apply the text augmentation function
    df[new_col_name] = df.apply(apply_mlm, axis=1)

    # Compute ROUGE scores
    rouge = Rouge()
    rouge_scores = rouge.get_scores(df[new_col_name], df["original_sentence"])
    rouge_df = pd.DataFrame([
        {
            'r1f1': score['rouge-1']['f'],
            'r2f1': score['rouge-2']['f'],
            'rlf1': score['rouge-l']['f'],
        }
        for score in rouge_scores
    ])

    # Compute SacreBLEU scores
    sacrebleu_scores = [sacrebleu.corpus_bleu([aug], [[orig]]).score for aug, orig in zip(df[new_col_name], df["original_sentence"])]
    df["sacrebleu_score"] = sacrebleu_scores

    # SBERT Score with Cosine Similarity
    df["sbert_score"] = [calculate_sbert_score(orig, aug) for orig, aug in zip(df["original_sentence"], df[new_col_name])]

    # BERTScore
    P, R, F1 = score(df[new_col_name], df["original_sentence"], lang="en", rescale_with_baseline=True)
    df["bertscore_f1"] = F1.tolist()


    # Combine the dataframes
    result_df = pd.concat([df, rouge_df], axis=1)
    result_df["method"] = "mlm3"

    return result_df

# Running on dataset

In [ ]:
file_path = '/content/MyDrive/MyDrive/Research/Thesis: BDA/Main/evaluation/(old)Youtube/Datasets/yt_sentiment_train_10.csv'

In [ ]:
result_df = mlm_and_evaluate_dataset(file_path, 'sentence1', 'augmented_sentence')

KeyboardInterrupt: 

In [ ]:
result_df

### Saving augmented dataset

In [ ]:
result_df.to_csv('/content/MyDrive/MyDrive/Research/Thesis: BDA/Main/evaluation/(old)Youtube/Datasets/yt_sentiment_train_10_mlm3.csv', index=False)